# Trabalho 2 - Erikson Ferreira

## Importações

In [28]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Geração de números aleatórios
import random

In [29]:
# Geração de gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import math
#progess bar
from tqdm import tqdm

###  Definição da Função de Rastringin

In [30]:
def rastrigin(x, y):
    return 20 + x**2 - 10 * math.cos(2 * math.pi * x) + y**2 - 10 * math.cos(2 * math.pi * y)

###  Hill Climbing

In [31]:
def hill_climbing():
    restarts = 20
    iterations = 1000
    restart_iter = 50
    df_hill_climbing_restart = pd.DataFrame()
    best_solution = None
    best_fitness = float('inf')
    last_best_coord = [0,0,0]
    # for restart in range(restarts):
    x = random.uniform(-5.12, 5.12)
    y = random.uniform(-5.12, 5.12)
    
    for iteration in range(iterations):
        current_fitness = rastrigin(x, y)
        # print(current_fitness)
        
        if current_fitness < best_fitness:
            best_solution = (x, y)
            last_best_coord = [x, y, current_fitness]
            best_fitness = current_fitness
            df_temp = pd.DataFrame(last_best_coord)
            df_hill_climbing_restart = pd.concat([df_hill_climbing_restart, df_temp.T], axis =0).reset_index(drop=True)
        else:
            df_temp = pd.DataFrame(last_best_coord)
            df_hill_climbing_restart = pd.concat([df_hill_climbing_restart, df_temp.T], axis =0).reset_index(drop=True)
        
        # Produzir vizinho
        delta_x = random.gauss(0, 0.1)
        delta_y = random.gauss(0, 0.1)
        neighbor_x = x + delta_x
        neighbor_y = y + delta_y
        
        if -5.12 <= neighbor_x <= 5.12 and -5.12 <= neighbor_y <= 5.12:
            neighbor_fitness = rastrigin(neighbor_x, neighbor_y)
            
            if neighbor_fitness < current_fitness:
                x = neighbor_x
                y = neighbor_y
    
        # Realizar restart
        if iteration % restart_iter == 0:
            x = random.uniform(-5.12, 5.12)
            y = random.uniform(-5.12, 5.12)
    
    return best_solution, df_hill_climbing_restart

### Simulated Annealing

In [32]:
import math
import random


def generate_random_solution():
    x = random.uniform(-5.12, 5.12)
    y = random.uniform(-5.12, 5.12)
    return (x, y)

def generate_neighbor(solution):
    x = solution[0] + random.gauss(0, 0.1)
    y = solution[1] + random.gauss(0, 0.1)
    return (x, y)

def simulated_annealing():
    initial_solution = generate_random_solution()
    current_solution = initial_solution
    best_solution = initial_solution
    current_fitness = rastrigin(current_solution[0], current_solution[1])
    best_fitness = current_fitness
    temperature = 1.0
    cooling_rate = 0.001
    iteration = 0
    last_best_coord = [0,0,0]
    df_simulated_annealing = pd.DataFrame()

    while iteration < 1000:
        # print(iteration)
        for _ in range(1000):
        
            neighbor_solution = generate_neighbor(current_solution)
            # print(_)
            if -5.12 <= neighbor_solution[0] <= 5.12 and -5.12 <= neighbor_solution[1] <= 5.12:
              
                neighbor_fitness = rastrigin(neighbor_solution[0], neighbor_solution[1])
                # print(random.random(), math.exp((current_fitness - neighbor_fitness) / max(temperature, 1e-10)))
                if neighbor_fitness < current_fitness or random.random() < math.exp((current_fitness - neighbor_fitness) / max(temperature, 1e-10)):
                    # print(iteration)
                    current_solution = neighbor_solution
                    current_fitness = neighbor_fitness
                    if neighbor_fitness < best_fitness:
                        best_solution = neighbor_solution
                        best_fitness = neighbor_fitness
                        last_best_coord = [best_solution[0],best_solution[1], neighbor_fitness]
                        df_temp = pd.DataFrame(last_best_coord)
                        df_simulated_annealing = pd.concat([df_simulated_annealing, df_temp.T], axis =0).reset_index(drop=True)
                        iteration += 1
                        break
                    else:
                        df_temp = pd.DataFrame(last_best_coord)
                        df_simulated_annealing = pd.concat([df_simulated_annealing, df_temp.T], axis =0).reset_index(drop=True)
                        iteration += 1
                        break
                else:
                   
                    df_temp = pd.DataFrame(last_best_coord)
                    df_simulated_annealing = pd.concat([df_simulated_annealing, df_temp.T], axis =0).reset_index(drop=True)
                    iteration += 1
                    break
                        
        if iteration > 999:
            break
        
        if iteration <= 900:
            temperature = 1.0 - (iteration / 900) * cooling_rate
        else:
            temperature = 0.0
        

    return best_solution, df_simulated_annealing



### Genetic Algorithm

In [113]:
def create_population(size):
    population = []
    for _ in range(size):
        x = random.uniform(-5.12, 5.12)
        y = random.uniform(-5.12, 5.12)
        population.append((x, y))
    return population

def evaluate_population(population):
    fitness_values = []
    for individual in population:
        fitness = rastrigin(individual[0], individual[1])
        fitness_values.append(fitness)
    return fitness_values

def select_parents(population, fitness_values):
    parents = random.choices(population, weights=fitness_values, k=2)
    return parents

def crossover(parents):
    alpha = random.uniform(0, 1)
    x1, y1 = parents[0]
    x2, y2 = parents[1]
    child1 = (x1 * alpha + x2 * (1 - alpha), y1 * alpha + y2 * (1 - alpha))
    child2 = (x2 * alpha + x1 * (1 - alpha), y2 * alpha + y1 * (1 - alpha))
    return child1, child2

def mutate(individual):
    x = individual[0] + random.gauss(0, 0.1)
    y = individual[1] + random.gauss(0, 0.1)
    x = max(-5.12, min(5.12, x))
    y = max(-5.12, min(5.12, y))
    return x, y

def genetic_algorithm():
    population_size = 20
    generations = 50
    mutation_prob = 0.3
    historico = []
    population = create_population(population_size)
    df_genetic_algorithm = pd.DataFrame()
    
    for _ in range(generations):
        fitness_values = evaluate_population(population)
        parents = select_parents(population, fitness_values)
        # print(population)
        new_population = []
        for _ in range(population_size // 2):
            child1, child2 = crossover(parents)
            # print(len(population))
            # Mutação
            if random.random() < mutation_prob:
                child1 = mutate(child1)
            if random.random() < mutation_prob:
                child2 = mutate(child2)
            
            new_population.append(child1)
            new_population.append(child2)
           
        
        population = new_population
        historico.append(population)
       
        
       
        # print(population)
    df_temp = pd.DataFrame([historico])
    df_genetic_algorithm = pd.concat([df_genetic_algorithm, df_temp.T], axis =0).reset_index(drop=True)
    
    best_solution = min(population, key=lambda x: rastrigin(x[0], x[1]))
    return best_solution, df_genetic_algorithm




#### Minimizando a função

In [115]:
solution_hill_climbing, df_hill_climbing = hill_climbing()
solution_hill_climbing, df_hill_climbing

((0.994362582998376, 0.999147116516275),
             0         1          2
 0    2.433905 -2.807311  39.431249
 1   -0.248672 -1.772442  21.714574
 2   -0.225338 -1.761038  20.915734
 3   -0.225338 -1.761038  20.915734
 4   -0.231071 -1.882742  15.005031
 ..        ...       ...        ...
 995  0.994363  0.999147   1.993468
 996  0.994363  0.999147   1.993468
 997  0.994363  0.999147   1.993468
 998  0.994363  0.999147   1.993468
 999  0.994363  0.999147   1.993468
 
 [1000 rows x 3 columns])

In [120]:
df_hill_climbing.columns = ['x', 'y', 'z']
df_hill_climbing


,x,y,z
0,2.433905,-2.807311,39.431249
1,-0.248672,-1.772442,21.714574
2,-0.225338,-1.761038,20.915734
3,-0.225338,-1.761038,20.915734
4,-0.231071,-1.882742,15.005031
...,...,...,...
995,0.994363,0.999147,1.993468
996,0.994363,0.999147,1.993468
997,0.994363,0.999147,1.993468
998,0.994363,0.999147,1.993468


In [116]:
solution_simulated_annealing, df_simulated_annealing = simulated_annealing()
solution_simulated_annealing, df_simulated_annealing

((-3.983891016966922, 3.9728034291445633),
             0         1          2
 0   -3.630715  4.474669  69.891006
 1   -3.630715  4.474669  69.891006
 2   -3.705248  4.480654  66.506290
 3   -3.707954  4.356251  61.528046
 4   -3.707954  4.356251  61.528046
 ..        ...       ...        ...
 995 -3.983891  3.972803  31.851381
 996 -3.983891  3.972803  31.851381
 997 -3.983891  3.972803  31.851381
 998 -3.983891  3.972803  31.851381
 999 -3.983891  3.972803  31.851381
 
 [1000 rows x 3 columns])

In [123]:
df_simulated_annealing.columns = ['x', 'y', 'z']
df_simulated_annealing

,x,y,z
0,-3.630715,4.474669,69.891006
1,-3.630715,4.474669,69.891006
2,-3.705248,4.480654,66.506290
3,-3.707954,4.356251,61.528046
4,-3.707954,4.356251,61.528046
...,...,...,...
995,-3.983891,3.972803,31.851381
996,-3.983891,3.972803,31.851381
997,-3.983891,3.972803,31.851381
998,-3.983891,3.972803,31.851381


In [118]:
solution_genetic_algorithm, df_genetic_algorithm = genetic_algorithm()
solution_genetic_algorithm, df_genetic_algorithm

((-4.393722299173211, 4.300637494846601),
                                                     0
 0   [(-3.268211830974277, 2.223790945171829), (-2....
 1   [(-3.7198345834012887, 4.104765118511535), (-3...
 2   [(-3.7516671910667707, 4.22102660530544), (-3....
 3   [(-3.8235444794679427, 4.395116008789929), (-3...
 4   [(-3.8300767501164534, 4.281687046678535), (-3...
 5   [(-3.8325058740161957, 4.195694207888738), (-3...
 6   [(-3.9926946932126732, 4.245756251754147), (-3...
 7   [(-3.7367874191219026, 4.031170801114105), (-3...
 8   [(-3.8249751336773854, 4.1882926448929645), (-...
 9   [(-3.9606756799203726, 4.221299917271176), (-3...
 10  [(-3.9222034416348857, 4.207218111184822), (-3...
 11  [(-3.899674228147126, 4.1989718528502165), (-3...
 12  [(-3.900489543276303, 4.199270278677774), (-3....
 13  [(-3.817265473278616, 4.173855159737357), (-3....
 14  [(-3.900626825374462, 4.199320527375836), (-3....
 15  [(-3.7148386038871406, 4.320396051291465), (-4...
 16  [(-3.9798681881534

In [125]:
df_genetic_algorithm.columns = ['population']
df_genetic_algorithm

,population
0,"[(-3.268211830974277, 2.223790945171829), (-2...."
1,"[(-3.7198345834012887, 4.104765118511535), (-3..."
2,"[(-3.7516671910667707, 4.22102660530544), (-3...."
3,"[(-3.8235444794679427, 4.395116008789929), (-3..."
4,"[(-3.8300767501164534, 4.281687046678535), (-3..."
5,"[(-3.8325058740161957, 4.195694207888738), (-3..."
6,"[(-3.9926946932126732, 4.245756251754147), (-3..."
7,"[(-3.7367874191219026, 4.031170801114105), (-3..."
8,"[(-3.8249751336773854, 4.1882926448929645), (-..."
9,"[(-3.9606756799203726, 4.221299917271176), (-3..."


##### Box Plots

In [37]:
# df_custo_T = df_custo.T
# df_custo_T.describe()

In [38]:
# px.box(df_custo_T, points='all', y=df_custo_T.columns.tolist()).update_layout(
#     xaxis_title="Algoritmo", yaxis_title="Custo", title="Custo por Algoritmo")

In [40]:
import plotly.graph_objects as go
import numpy as np

def plot_rastrigin():
    x = np.linspace(-5.12, 5.12, 100)
    y = np.linspace(-5.12, 5.12, 100)
    X, Y = np.meshgrid(x, y)
    Z = 20 + X**2 - 10 * np.cos(2 * np.pi * X) + Y**2 - 10 * np.cos(2 * np.pi * Y)
    
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, opacity=0.1)])
    fig.update_layout(title='Rastrigin Function', autosize=False,
                      scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')),
                      width=800, height=800)
    # fig.update_traces(contours_z=dict(show=True, usecolormap=True,
    #                               highlightcolor="limegreen", project_z=True))
    fig.add_trace(go.Scatter3d(x=[hill_climbing()[0]],
                 y=[hill_climbing()[1]],
                 z=[rastrigin(hill_climbing()[0], hill_climbing()[1])],
                 ))
    fig.show()

# Plot da função de Rastrigin
plot_rastrigin()

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

In [ ]:
# Executa N vezes para gerar estatísticas da variável custo
# Executa N vezes para gerar estatísticas da variável custo
def executa_n_vezes_trab2(algoritmos):

    
    # Cria DataFrame para armazenar os resultados
 
    df_custo = cria_df_custos(algoritmos, n_vezes)
    df_history = pd.DataFrame(columns=algoritmos.keys())
    df_hill_climbing = pd.DataFrame()
    df_hill_climbing_restart = pd.DataFrame()
    df_genetic_algorithm = pd.DataFrame()
    df_genetic_algorithm_population = pd.DataFrame()
    df_simulated_annealing = pd.DataFrame()
    for algoritmo, funcao_algoritmo in algoritmos.items():

        print(algoritmo)

        for i, count in zip(range(n_vezes), tqdm(range(n_vezes))):
            df = None
            
            solucao = funcao_algoritmo()
            df_temp = pd.DataFrame(solucao)
         
            
            if algoritmo == 'Hill-Climbing':
                df_hill_climbing = pd.concat([df_hill_climbing, df_temp.T], axis =0).reset_index(drop=True)
            
            elif algoritmo == 'Hill-Climbing Restart':
                df_hill_climbing_restart = pd.concat([df_hill_climbing_restart, df_temp.T], axis =0).reset_index(drop=True)
            
            elif algoritmo == 'Genetic Algorithm':
                df_genetic_algorithm = pd.concat([df_genetic_algorithm, df_temp.T], axis =0).reset_index(drop=True)
                # df  = pd.DataFrame([historico])
                # df_genetic_algorithm_population = pd.concat([df_genetic_algorithm_population, df], axis =0).reset_index(drop=True)

            elif algoritmo == 'Simulated Annealing':
                df  = pd.DataFrame([historico[1]], columns=historico[1].keys())
                df_simulated_annealing = pd.concat([df_simulated_annealing, df], axis =0).reset_index(drop=True)
                df_simulated_annealing_history = pd.concat([df_simulated_annealing_history, df_temp.T], axis =0).reset_index(drop=True)
            
    
            df_custo.loc[algoritmo,i] = custo
     
            # else:
                # custo, solucao = funcao_algoritmo(df_coord)
                # df_custo.loc[algoritmo,i] = custo

            # print(f'{custo:10.3f}  {solucao}')

    return df_custo, df_history, df_hill_climbing, df_hill_climbing_restart, df_simulated_annealing, df_simulated_annealing_history, df_genetic_algorithm, df_genetic_algorithm_population



In [ ]:
# Dicionario com Nomes dos modelos e suas respectivas variantes
# Tuple: (Algoritmo, Variante): funcao_algoritmo
algoritmos = {
    
    'Hill-Climbing': hill_climbing,
    'Hill-Climbing Restart': hill_climbing_restart,
    'Simulated Annealing': simulated_annealing,
    'Genetic Algorithm': tsp_genetic,
    
}

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_rastrigin():
    x = np.linspace(-5.12, 5.12, 100)
    y = np.linspace(-5.12, 5.12, 100)
    X, Y = np.meshgrid(x, y)
    Z = 20 + X**2 - 10 * np.cos(2 * np.pi * X) + Y**2 - 10 * np.cos(2 * np.pi * Y)
    
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
    fig.update_layout(title='Rastrigin Function', autosize=False,
                      scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Z')),
                      width=800, height=800)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
    
    fig.show()

# Plot da função de Rastrigin
plot_rastrigin()